In [9]:
import matplotlib.pyplot as plt
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Reshape, Conv2DTranspose, Lambda
from keras.models import Model
import numpy as np
# import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
mnist_train = mnist.train.images
mnist_test = mnist.test.images
mnist_validation = mnist.validation.images

print(mnist_train.shape)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
(55000, 784)


In [10]:
# Model
# Training Parameters
learning_rate = 0.001
num_steps = 30000
batch_size = 256

display_step = 1000

# Network Parameters
num_hidden_1 = 256 # number of parameters in the first fully connected layer in encoder and decoder
num_hidden_2 = 128 # number of parameters in the second fully connected layer in encoder
num_hidden_3 = 3136 # number of parameters in the second fully connected layer in decoder
num_input = 784 # MNIST data input (img shape: 28*28)

train_on_gpu = True # True for training on GPU, False for training on CPU

# Build computational graph using Keras

X = Input(shape = (28, 28, 1)) # placeholder for input data

# Function that builds computational graph of encoder
def encoder(x):  
    conv1 = Conv2D(32, (5, 5), activation='relu', padding='same')(x)   
    
    pool1 = MaxPooling2D((2, 2), strides = 2, padding='same')(conv1)
        
    conv2 = Conv2D(64, (5, 5), activation='relu', padding='same')(pool1)
    
    pool2 = MaxPooling2D((2, 2), strides = 2, padding='same')(conv2)
    
    flattened = Flatten()(pool2)
    
    dense1 = Dense(num_hidden_1, activation = 'relu')(flattened)
        
    dense2 = Dense(num_hidden_2, activation = 'sigmoid')(dense1)
    
    return dense2

# Function that builds computational graph of decoder
def decoder(x):
    dense1 = Dense(num_hidden_1, activation = 'relu')(x)
    
    dense2 = Dense(num_hidden_3, activation = 'relu')(dense1)
        
    dense2 = Reshape((7, 7, 64))(dense2)
        
    deconv1 = Conv2DTranspose(64, (5, 5), activation='relu', padding='same')(dense2)
    
    upsample1 = UpSampling2D((2, 2))(deconv1)
            
    deconv2 = Conv2DTranspose(32, [5, 5], activation='relu', padding='same')(upsample1)
        
    upsample2 = UpSampling2D((2, 2))(deconv2)
            
    flattened = Flatten()(upsample2)
        
    dense3 = Dense(28 * 28, activation = 'sigmoid')(flattened)
        
    dense3 = Reshape((28, 28, 1))(dense3)
    
    return dense3

# Autoencoder model
encoder_op = Lambda(encoder)(X)
decoder_op = Lambda(decoder)(encoder_op)
autoencoder = Model(X, decoder_op)        

# Determine the optimizer and the loss function
autoencoder.compile(optimizer='RMSprop', loss='mean_squared_error')


In [ ]:
# Training
mnist_train = np.reshape(mnist_train, (mnist_train.shape[0], 28, 28, 1))
mnist_test = np.reshape(mnist_test, (mnist_test.shape[0], 28, 28, 1))

autoencoder.fit(mnist_train, mnist_train, num_steps, batch_size, shuffle=True, validation_data=(mnist_test, mnist_test))


Train on 55000 samples, validate on 10000 samples
Epoch 1/256


In [ ]:
# Test  
n = 5 # number of images to test
_, axarr = plt.subplots(2, n) # create axis array to display images
    
# Encode and decode images from test set and visualize their reconstruction.
batch_x, _ = mnist.test.next_batch(n)
batch_x = np.reshape(batch_x, [-1, 28, 28, 1])

g = autoencoder.predict(batch_x) 

for i in range(n):       
    axarr[0, i].imshow(batch_x[i].reshape([28, 28]), origin="upper", cmap="gray")
    axarr[1, i].imshow(g[i].reshape([28, 28]), origin="upper", cmap="gray")